# Text Mining the Erowid Experience Vault

#### import packages and set path

In [3]:
import os, sys, re, math
path = os.path.abspath("")+"/"
print path

C:\Users\Glenn\Documents\GitHub\mining-erowid/


### generator to iterate files into memory as lists of words

In [4]:
import nltk, nltk.tokenize, nltk.corpus
from bs4 import BeautifulSoup

def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

def is_okay_word(s):
    import re
    if len(s)==1:
        return False
    elif is_number(s) and float(s)<1900:
        return False
    elif re.match('\d+[mM]?[gGlLxX]',s):
        return False
    elif re.match('\d+[oO][zZ]',s):
        return False
    else:
        return True

def yield_body_text(path):
    stopwords = nltk.corpus.stopwords.words('english')
    tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
    tagger = nltk.tag.UnigramTagger(nltk.corpus.brown.tagged_sents())
    lemmatizer = nltk.WordNetLemmatizer()
    from nltk.corpus import wordnet
    experiences = os.listdir(path+'xml')
    for n, experience in enumerate(experiences):
        #if n<9900:
            #continue
        #if n>10000:
            #return
        words = []
        with open(path+"xml/"+experience) as file:
            soup = BeautifulSoup(file)
            #print experience
            tokens = tokenizer.tokenize(soup.bodytext.contents[0])
            pos = tagger.tag(tokens)
            words = []
            for token in pos:
                if token[1] == 'NN':
                    pos = wordnet.NOUN
                elif token[1] == 'JJ':
                    pos = wordnet.ADJ
                elif token[1] == 'VB':
                    pos = wordnet.VERB
                elif token[1] == 'RV':
                    pos = wordnet.ADV
                else:
                    pos = wordnet.NOUN
                lemma = lemmatizer.lemmatize(token[0], pos)
                if is_okay_word(lemma) and lemma not in stopwords:
                    words.append(lemma)
        if n%1000==0:
            print("Finished " + str(n) + " files out of " + str(len(experiences)))
        yield " ".join(words)

## create the bag-of-words using CountVectorizer

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(min_df=2)
bag = cv.fit_transform(yield_body_text(path))


Finished 0 files out of 19634
Finished 1000 files out of 19634
Finished 2000 files out of 19634
Finished 3000 files out of 19634
Finished 4000 files out of 19634
Finished 5000 files out of 19634
Finished 6000 files out of 19634
Finished 7000 files out of 19634
Finished 8000 files out of 19634
Finished 9000 files out of 19634
Finished 10000 files out of 19634
Finished 11000 files out of 19634
Finished 12000 files out of 19634
Finished 13000 files out of 19634
Finished 14000 files out of 19634
Finished 15000 files out of 19634
Finished 16000 files out of 19634
Finished 17000 files out of 19634
Finished 18000 files out of 19634
Finished 19000 files out of 19634


#### Save or load the serialized matrix

In [4]:
import pickle
pickle.dump(bag, open("bag.p","wb"))
#bag = pickle.load(open("bag.p","rb"))

### Determine feature weights

In [6]:
from sklearn.feature_extraction.text import TfidfTransformer
tf = TfidfTransformer()
tfidf_bag = tf.fit_transform(bag)

### Model topics

In [7]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=100)
svd_bag = svd.fit_transform(tfidf_bag)

#### Get lists of important terms for each topic

In [9]:
import numpy as np
def get_topic_names(model, vzr):
    topics = []
    for i in range(np.shape(model.components_)[0]):
        weights = list(model.components_[i])
        z = zip(weights,vzr.get_feature_names())
        z.sort(reverse=True)
        unzip = zip(*z)
        topics.append(unzip[1])
    return topics

topics = get_topic_names(svd,cv)

    

In [10]:
len(topics)

100

In [11]:
topics[0]

(u'wa',
 u'like',
 u'time',
 u'felt',
 u'the',
 u'friend',
 u'would',
 u'it',
 u'one',
 u'trip',
 u'experience',
 u'could',
 u'we',
 u'back',
 u'feel',
 u'feeling',
 u'really',
 u'around',
 u'hour',
 u'get',
 u'started',
 u'thing',
 u'thought',
 u'day',
 u'my',
 u'effect',
 u'got',
 u'drug',
 u'still',
 u'went',
 u'much',
 u'go',
 u'first',
 u'took',
 u'night',
 u'room',
 u'this',
 u'minute',
 u'body',
 u'didn',
 u'good',
 u'little',
 u'people',
 u'going',
 u'life',
 u'everything',
 u'way',
 u'decided',
 u'began',
 u'eye',
 u'know',
 u'mind',
 u'even',
 u'something',
 u'take',
 u'think',
 u'see',
 u'seemed',
 u'never',
 u'point',
 u'high',
 u'also',
 u'looked',
 u'remember',
 u'two',
 u'pill',
 u'said',
 u'after',
 u'next',
 u'salvia',
 u'hit',
 u'though',
 u'made',
 u'house',
 u've',
 u'he',
 u'so',
 u'music',
 u'came',
 u'well',
 u'bit',
 u'told',
 u'tried',
 u'light',
 u'head',
 u'another',
 u'sleep',
 u'bed',
 u'anything',
 u'at',
 u'come',
 u'make',
 u'mushroom',
 u'try',
 u'ever'

In [ ]:
#I can actually use gensim now...
def doctext(path):
		import nltk, nltk.tokenize
		tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
		lemmatizer = nltk.WordNetLemmatizer()
		all_xml = os.listdir(path+'xml');
		experiences = [filename for filename in all_xml if re.match('\d+.xml',filename)]
		for n, experience in enumerate(experiences):
			#if n>100:
				#return
			with open(path+'xml/'+experience) as file:
				soup = BeautifulSoup(file)
				text = soup.bodytext.contents[0]
			if n%100==0:
				print("Finished " + str(n) + " files out of " + str(len(experiences)))
			yield text

def is_number(s):
	try:
		float(s)
		return True
	except ValueError:
		return False
		
def is_okay_word(word):
	import re
	if len(word)==1:
		return False
	elif is_number(word) and float(word)<1900:
		return False
	elif re.match('\d+[mM]?[gGlLxX]',word):
		return False
	elif re.match('\d+[oO][zZ]',word):
		return False
	else:
		return True

def doctext(path):
		from nltk.corpus import stopwords
		import nltk, nltk.tokenize
		tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
		lemmatizer = nltk.WordNetLemmatizer()
		stopw = stopwords.words('english')
		all_xml = os.listdir(path+'xml');
		experiences = [filename for filename in all_xml if re.match('\d+.xml',filename)]
		for n, experience in enumerate(experiences):
			#if n>10:
				#return
			words = []
			with open(path+'xml/'+experience) as file:
				soup = BeautifulSoup(file)
				tokens = tokenizer.tokenize(soup.bodytext.contents[0])
				lemmas = [lemmatizer.lemmatize(token.lower()) for token in tokens]
				for lemma in lemmas:
					if is_okay_word(lemma) and lemma not in stopw:
						words.append(lemma)	
			if n%100==0:
				print("Finished " + str(n) + " files out of " + str(len(experiences)))
			yield words

if True:
	from gensim import corpora, models, similarities
	mypath = "/home/glenn/Desktop/brainclouds/"
	dictionary = corpora.Dictionary()
	corpus = [dictionary.doc2bow(text, allow_update=True) for text in doctext(mypath)]
	dictionary.save(mypath+'bc_dict.dict')
	corpora.MmCorpus.serialize(mypath+'bc_corpus.mm', corpus)
	
if True:
	from gensim import corpora, models, similarities
	mypath = "/home/glenn/Desktop/brainclouds/"
	corpus = corpora.MmCorpus(mypath+'bc_corpus.mm')
	dictionary = corpora.Dictionary.load(mypath+'bc_dict.dict')
	
if True:
	tfidf = models.TfidfModel(corpus)
	corpus_tfidf = tfidf[corpus]

if True:
	lsi= models.ldamodel.LdaModel(corpus_tfidf, num_topics=100, id2word=dictionary)
	corpus_lsi = lsi[corpus_tfidf]

if True:
	lsi10 = models.ldamodel.LdaModel(corpus_tfidf, num_topics=10, id2word=dictionary)
	corpus_lsi10 = lsi10[corpus_tfidf]
	
if True:
	lda = models.ldamodel.LdaModel(corpus_tfidf, num_topics=100, id2word=dictionary)
	corpus_lda = lda[corpus_tfidf]
	
if True:
	hdp = models.hdpmodel.HdpModel(corpus_tfidf, id2word=dictionary)
	corpus_hdp = hdp(corpus_tfidf)
	
if True:
	corpora.MmCorpus.serialize(mypath+'bc_tfidf.mm', corpus_tfidf)
	corpora.MmCorpus.serialize(mypath+'bc_lsi.mm', corpus_lsi)
	lsi10.save(mypath+'model.lsi10')
	lsi.save(mypath+'model.lsi')
	tfidf.save(mypath + 'model.tfidf')
	
	
	from gensim import corpora, models, similarities
	mypath = "/home/glenn/Desktop/brainclouds/"
	corpus = corpora.MmCorpus(mypath+'bc_corpus.mm')
	dictionary = corpora.Dictionary.load(mypath+'bc_dict.dict')+'model.tfidf')
	
if True:
	from gensim import corpora, models, similarities
	mypath = "/home/glenn/Desktop/brainclouds/"
	corpus = corpora.MmCorpus(mypath+'bc_corpus.mm')
	dictionary = corpora.Dictionary.load(mypath+'bc_dict.dict')
	corpus_tdif = corpora.MmCorpus(mypath+'bc_corpus.mm')
	tfidf = models.TfidfModel.load(mypath + 'model.tfidf')
	lsi = models.LsiModel.load(mypath+'model.lsi')

if True:
	import gensim.matutils
	corpus_matrix = gensim.matutils.corpus2csc(corpus).T
	tfidf_matrix = gensim.matutils.corpus2csc(corpus_tfidf).T
	lsi_matrix = gensim.matutils.corpus2csc(corpus_lsi).T
	
if True:
	from sklearn.cluster import DBSCAN
	from sklearn.preprocessing import StandardScaler
	lsi_scaled = StandardScaler(with_mean=False).fit_transform(lsi_matrix)
	db = DBSCAN().fit(lsi_scaled)
	core_samples = db.core_sample_indices
	labels = db.labels_